# Co-locating Temperature Anomaly Values between CBP Monitoring Stations and Satellite SST

## Notebook Overview
### Tasks
- Select satellite SST and SST anomaly values for each location and date in which there is an in situ observation from the Chesapeake Bay Program
- Save two csvs of the results: one csv for in situ and satellite SSTs and a second csv for the in situ and satellite SST anomalies

This script creates a csv which holds the corresponding validation data points from both Chesapeake Bay Program's (CBP) Water Quality dataset and the two satellite datasets. The csv contains one row for each day and pixel in which there was a CBP validation point, and either a MUR or a Geopolar satellite pixel.
Output: csv of temperature values that can be observed in the satellite SST datasets and the in situ CBP data

## Analysis

In [1]:
import os
import warnings

import xarray as xr
import pandas as pd
import numpy as np

In [2]:
REPO_ROOT = '/Users/rwegener/repos/chesapeake_mhw'
SAVE_FIGS = False

## Read CBay Program In Situ Anomaly Data

In [3]:
# OLD NAMING CONVENTION:
# path = os.path.join(REPO_ROOT, 'data/interim', 'cbp_stations_climatology_anomaly_filtered.csv')
# NEW NAMING CONVENTION: 'cbp_temps_selected_stations_with_climatology.csv'
path_anom = os.path.join(REPO_ROOT, 'data/interim', 'cbp_temps_selected_stations_with_climatology.csv')
anom_raw = pd.read_csv(path_anom, parse_dates=[1])

In [4]:
path_sst = os.path.join(REPO_ROOT, 'data/interim', 'cbp_stations_climatology_raw_filtered.csv')
sst_raw = pd.read_csv(path_sst, parse_dates=[1])

## Helper Functions

Creating functions for repeated tasks

In [5]:
def get_satellite_sst(full_sst, lat, lon, time):
    '''
    For a given latitude, longitude, and time extract the SST value at that location
    in the give satellite dataset.
    '''
    # time does NOT have nearest interpolation because we do not want adjacent days to
    # be selected
    try:
        matching_array = full_sst.sel(lat=lat, lon=lon, 
                                      method='nearest').sel(time=time.strftime('%Y-%m-%d')).values
        if matching_array.size == 1:
            matching_sst = matching_array[0]
        else:
            matching_sst = np.nan
            warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
    except KeyError:
        # If a key error was raised the corresponding date was not found. Return nan for that sample location
        matching_sst = np.nan
    # print('returning ', matching_sst)
    return matching_sst

In [8]:
sst_raw.columns

Index(['Station', 'SampleDate', 'MeasureValue', 'Latitude', 'Longitude'], dtype='object')

In [9]:
anom_raw.columns

Index(['Station', 'SampleDate', 'sst_cbp', 'Latitude', 'Longitude', 'clim_cbp',
       'anom_cbp'],
      dtype='object')

## Initialize the output dataframe

In [10]:
anom_raw = anom_raw[['Station', 'Latitude', 'Longitude', 'SampleDate', 'anom_cbp']]
wq_anom = anom_raw.copy()
wq_anom['anom_geopolar'] = -999
wq_anom['anom_mur'] = -999
wq_anom['anom_ostia'] = -999

In [11]:
sst_raw = sst_raw[['Station', 'Latitude', 'Longitude', 'SampleDate', 'MeasureValue']]
wq_sst = sst_raw.copy()
wq_sst['geopolar'] = -999
wq_sst['mur'] = -999
wq_sst['ostia'] = -999

## Extract SST value corresponding to CBP In situ observations

### Geo-Polar SST

In [12]:
# Open raw SST
path = os.path.join(
    REPO_ROOT, 'data/raw', 
    'L4_GHRSST-SSTfnd-Geo_Polar_Blended_Night-GLOB-v02.0-fv01.0_CB_20020901_20230831.nc'
)
geopolar = xr.open_dataset(path).analysed_sst
# convert kelvin to celsius & update metadata
geopolar.values = geopolar.values - 273.15
geopolar.attrs.update({'units': 'celsius',})

# Open calculated climatological SST
path = os.path.join(REPO_ROOT, 'data/interim', 'geopolar_climatology_chesapeake.nc')
geopolar_clim = xr.open_dataset(path).climatology

# Compute SST anomaly
geopolar_anom = geopolar - geopolar_clim

Timing Notes

Feb 1: ~22,000 rows: ~50 seconds


In [13]:
%%time

# Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_anom['anom_geopolar'] = wq_anom.apply(lambda x: get_satellite_sst(geopolar_anom, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2018-03-12
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2018-03-12
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2018-03-12
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2018-03-12
  warnings.warn('Matching arra

CPU times: user 22.5 s, sys: 498 ms, total: 23 s
Wall time: 23 s


In [14]:
# Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_sst['geopolar'] = wq_sst.apply(lambda x: get_satellite_sst(geopolar, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2017-02-01
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2017-02-01
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2017-02-01
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_780/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2017-02-01
  warnings.warn('Matching arra

### MUR SST

In [10]:
# Open raw SST
path = os.path.join(
    REPO_ROOT, 'data/raw', 
    'MUR-JPL-L4_GHRSST-SSTfnd-GLOB-v02.0-fv04.1-20020901_20230831.nc'
)
mur = xr.open_dataset(path).analysed_sst
# convert kelvin to celsius & update metadata
mur.values = mur.values - 273.15
mur.attrs.update({'units': 'celsius',})

# Open calculated climatological SST
path = os.path.join(REPO_ROOT, 'data/interim', 'mur_climatology_chesapeake.nc')
mur_clim = xr.open_dataset(path).climatology

# Compute SST anomaly
mur_anom = mur - mur_clim

In [11]:
%%time

# Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_anom['anom_mur'] = wq_anom.apply(lambda x: get_satellite_sst(mur_anom, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching 

CPU times: user 22.1 s, sys: 533 ms, total: 22.6 s
Wall time: 22.8 s


In [20]:
# Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_sst['mur'] = wq_sst.apply(lambda x: get_satellite_sst(mur, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching array length is not 1. Date is likely missing (?)' + time.strftime('%Y-%m-%d'))
/var/folders/nb/y7dp0ct55_58hrhrhlz5vbkm0000gn/T/ipykernel_5423/2577563597.py:15: UserWarning: Matching array length is not 1. Date is likely missing (?)2022-11-09
  warnings.warn('Matching 

### OSTIA SST

TODO -- kernel dies when I add this step. Come back and address this.

In [ ]:
# Open raw SST
path = os.path.join(
    REPO_ROOT, 'data/raw', 
    'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2_analysed_sst_77.47W-75.53W_36.78N-39.97N_2007-01-01-2023-09-01.nc'
)
ostia = xr.open_dataset(path).analysed_sst
ostia = ostia.rename({'latitude': 'lat', 'longitude': 'lon'})
# convert kelvin to celsius & update metadata
ostia.values = ostia.values - 273.15
ostia.attrs.update({'units': 'celsius',})

# Open calculated climatological SST
path = os.path.join(REPO_ROOT, 'data/interim', 'ostia_climatology_chesapeake.nc')
ostia_clim = xr.open_dataset(path).climatology

# Compute SST anomaly
ostia_anom = ostia - ostia_clim

In [ ]:
ostia

In [ ]:
# %%time

# # Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_anom['anom_ostia'] = wq_anom.apply(lambda x: get_satellite_sst(ostia_anom, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

### Cleaning Output

Remove rows that don't have corresponding observations from any of the satellites.

In [12]:
wq_anom.columns

Index(['Latitude', 'Longitude', 'SampleDate', 'anom_cbp', 'anom_geopolar',
       'anom_mur', 'anom_ostia'],
      dtype='object')

In [21]:
wq_sst.columns

Index(['MeasureValue', 'Latitude', 'Longitude', 'SampleDate', 'geopolar',
       'mur', 'ostia'],
      dtype='object')

In [13]:
wq_anom = wq_anom[(~wq_anom['anom_mur'].isnull()) | (~wq_anom['anom_geopolar'].isnull()) | \
                (~wq_anom['anom_ostia'].isnull())]

In [15]:
wq_anom

,Latitude,Longitude,SampleDate,anom_cbp,anom_geopolar,anom_mur,anom_ostia
0,39.44149,-76.02599,2003-01-15,-2.038462,-0.813166,-0.803528,-999
1,39.44149,-76.02599,2004-01-14,-2.838462,-1.020229,-0.585965,-999
2,39.44149,-76.02599,2005-01-13,1.761538,0.318968,1.116807,-999
3,39.44149,-76.02599,2006-01-12,1.461538,1.184868,-1.978795,-999
4,39.44149,-76.02599,2007-01-12,2.161538,1.744897,3.050197,-999
...,...,...,...,...,...,...,...
13157,38.84250,-76.53410,2018-02-15,-0.273684,0.177036,NaN,-999
13158,38.84250,-76.53410,2019-02-11,0.026316,-0.566096,NaN,-999
13159,38.84250,-76.53410,2020-02-13,3.126316,1.065957,NaN,-999
13160,38.84250,-76.53410,2021-02-08,-0.573684,0.484139,NaN,-999


In [22]:
wq_sst = wq_sst[(~wq_sst['mur'].isnull()) | (~wq_sst['geopolar'].isnull())]

### Save File

In [ ]:
path = os.path.join(
    REPO_ROOT, 'data/processed', 
    'SSTanom_satellites_cbp_stations.csv'
    # 'anomaly_values_satellites_CBPstations_filtered.csv' OLD NAMING PATTERN
)
wq_anom.to_csv(path, index=False)

In [ ]:
wq_anom

In [ ]:
path = os.path.join(
    REPO_ROOT, 'data/processed', 
    'SST_satellites_cbp_stations.csv'
    # New name: sst_values_satellites_cbp_stations.csv
)
wq_sst.to_csv(path, index=False)

In [23]:
wq_sst

,MeasureValue,Latitude,Longitude,SampleDate,geopolar,mur,ostia
0,8.300,38.40000,-77.32000,2007-03-22,4.000000,NaN,-999
1,26.200,38.40000,-77.32000,2007-06-18,21.970001,NaN,-999
2,14.500,38.40000,-77.32000,2007-10-29,18.019989,NaN,-999
3,5.200,38.40000,-77.32000,2007-12-10,10.750000,NaN,-999
4,4.900,38.40000,-77.32000,2008-02-04,3.670013,NaN,-999
...,...,...,...,...,...,...,...
50988,29.460,37.50465,-76.79252,2008-07-22,NaN,27.756989,-999
50989,26.455,37.50465,-76.79252,2008-08-22,NaN,26.091003,-999
50990,25.105,37.50465,-76.79252,2008-09-17,NaN,23.483002,-999
50991,21.134,37.50465,-76.79252,2008-10-16,NaN,21.225006,-999


### Inspecting the remaining observations

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax2.set_ylim([800, 4000])

histogram = wq.SampleDate.hist(bins=68, ax=ax1, grid=False)
ax1.set_ylabel('Number of observations (3 month bins)')
ax1.set_xlabel('Year')
# ax1.grid(axis='y', linestyle='--', color='lightgrey')
# ax1.set_yticks(np.arange(100, 900, 100))

# Making the line plot
# didn't work using default pandas .plot(ax=ax2) I think because date range was off
avgs = wq.groupby(wq.SampleDate.dt.year).count().Station.values
year_start = pd.date_range('2003', '2019', freq='YS')
year_end = pd.date_range('2003', '2020', freq='Y')
# ax2.hlines(avgs, year_start, year_end, color='black')
ax2.set_ylabel('Average # of observations per calendar year')
# ax2.set_yticks(np.arange(1200, 3200, 3050))

fig.suptitle('Number of Available Validation Data Points over Time')

if SAVE_FIGS:
    path = os.path.join(REPO_ROOT, 'figures/supplemental', 'validation_points_temporal_histogram.png')
    plt.savefig(path)

## For Depth Sensitivity (RMSE) (delete this? or move to another file?) NOTE wq_sst should now be wq_anom if revisiting this (name was changed above)

Computing RMSE here so I don't have to read in file in another notebook

In [ ]:
wq_sst['geopolar_diff'] = wq_sst['geopolar_anom'] - wq_sst['MeasureAnomaly']
wq_sst['mur_diff'] = wq_sst['mur_anom'] - wq_sst['MeasureAnomaly']

In [ ]:
N = len(wq_sst[~wq_sst['geopolar_diff'].isnull()])

rmse_geopolar = np.sqrt((wq_sst['geopolar_diff']**2).sum() / N)
print('rmse geopolar: ', rmse_geopolar)

N = len(wq_sst[~wq_sst['mur_diff'].isnull()])

rmse_mur = np.sqrt((wq_sst['mur_diff']**2).sum() / N)
print('rmse mur: ', rmse_mur)

| Depth Range    | Geopolar RMSE | MUR RMSE | 
| -------- | ------- | ------- |
| 0.5-3m  | 1.422   | 1.768 |
| 0.5-7m | 1.477   | 1.763 |
| **1-3m**    |  1.38   | 1.75|
| 1-7m |1.4749  | 1.757  |

| Depth Range    | Geopolar RMSE | MUR RMSE | 
| -------- | ------- | ------- |
| 0.5-3m  | 1.42   | 1.77 |
| 0.5-7m | 1.48   | 1.76 |
| **1-3m**    |  1.38   | 1.75|
| 1-7m |1.47  | 1.76  |